In [24]:
import pandas as pd
from scipy.stats import shapiro
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [4]:
df=pd.read_csv("data//clean//vehiculos.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14631 entries, 0 to 14657
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Marca                   14631 non-null  object 
 1   Modelo                  14631 non-null  object 
 2   Precio                  14631 non-null  float64
 3   Potencia                14631 non-null  float64
 4   Tipo vendedor           14631 non-null  object 
 5   Categoría               14631 non-null  object 
 6   Tipo de vehículo        14631 non-null  object 
 7   puertas                 14631 non-null  int64  
 8   Versión del país        14631 non-null  object 
 9   Garantía                14631 non-null  int64  
 10  Kilometraje             14631 non-null  float64
 11  Año                     14631 non-null  int64  
 12  Tipo de cambio          14631 non-null  object 
 13  Capacidad               14631 non-null  float64
 14  Consumo de combustible  14631 non-null  flo

Como 'Versión del país' siempre contiene el valor 'España' no es relevante y lo podemos eliminar.

In [11]:
df=df.drop('Versión del país', axis=1)

Definimos las columnas numéricas y las categóricas:

In [12]:
num_cols=df.select_dtypes(['int64', 'float64']).columns
cat_cols=df.select_dtypes(['object']).columns

De las columnas numéricas hay que mirar cuales tienen distribución gausiana, para saber a cuales podemos estandarizar con el StandartScaler.

In [19]:
def gaussian_columns(df):
    gaus_cols=[]
    for column in df.columns:
        stat, p = shapiro(df[column])
        if p > 0.05:
            gaus_cols.append(column)
    return gaus_cols

In [22]:
df_num=df[num_cols]
gaus_cols=gaussian_columns(df_num)
print(gaus_cols)

[]


c:\Users\mabi1\anaconda3\envs\proj\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Ninguna es probable que tenga distribución gausiana. Ahora de las columnas numéricas vamos a separar las que tienen outliers de las que no, para ello cogeremos el dataframe con solo las columnas numéricas, tomaremos también el primer cuantil, el tercero y la diferencia entre ambos y con una fórmula matemática seleccionaremos las columnas donde hay outliers. Una vez obtenidas las columnas con outliers haremos un drop de las mismas en el dataframe de columnas numéricas y las restantes serán las que no tienen outliers. Es importante tener en cuenta cuales tienen outliers y cuales no para saber qué tipo de escalado aplicarles.

In [17]:
q1 = df_num.quantile(0.25)
q3 = df_num.quantile(0.75)
diff = q3 - q1
out_cols=df_num.columns[((df_num < (q1 - 1.5 * diff)) |(df_num > (q3 + 1.5 * diff))).any(axis=0)]
no_out_cols=df_num.drop(out_cols, axis=1).columns

In [18]:
print(out_cols)
print(no_out_cols)

Index(['Precio', 'Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año',
       'Capacidad', 'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')
Index(['Mes', 'CP'], dtype='object')


Así pues, ahora tenemos las columnas de nuestro dataset divididas en tres sets: columnas categóricas (cat_cols), columnas numéricas con outliers(out_cols) y columnas numéricas sin outliers (no_out_cols).
A las columnas categóricas les aplicaremos OneHotEncoder, a las que tienen outliers RobustScaler y a las que no tienen outliers MinMaxScaler.

In [26]:
out_pipeline = Pipeline(steps=[
    ('scale',RobustScaler())
])

no_out_pipeline = Pipeline(steps=[
    ('scale',MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('scale',OneHotEncoder())
])

Añadiremos estas pequeñas pipeslines a un ColumnTransformer para que a cada tipo de columna se le aplique lo indicado. Le indicaremos remainder='drop' para que ignore cualquier columna del dataframe que no esté contemplada en esas categorías. Y n_jobs= -1 para que use todos los procesadores que tengamos en paralelo.

In [27]:
col_trans = ColumnTransformer(transformers=[
    ('out_pipeline',out_pipeline,out_cols),
    ('no_out_pipeline',no_out_pipeline,no_out_cols),
    ('cat_pipeline', cat_pipeline, cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

Este column transformer será añadido a un pipeline final que también contendrá el modelo. Pero primero deberíamos decidir qué modelo utilizar.